In [16]:
import numpy 
import pandas as pd
import asyncio
import struct
import datetime
import nest_asyncio
import bleak
from bleak import BleakScanner, BleakClient

nest_asyncio.apply()

Функция расшифровки адверсайзмент данных:

In [12]:
def adv_decrypt(data):
    # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Уровень топлива: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Напряжение встроенной батареи: {battery_voltage} В")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Температура: {TD_temp_raw} Градус Цельсия")

    # Расшифровка версии прошивки
    TD_version_raw = int.from_bytes(data[5:6], byteorder='little', signed=False)
    print(f"Версия прошивки: {TD_version_raw}")
                    
    # Расшифровка периода (непосредственно измеренный параметр)
    TD_period_param = int.from_bytes(data[6:10], byteorder='little', signed=False)
    print(f"Период: {TD_period_param} ")

    # Расшифровка режима работы
    TD_mode_raw = int.from_bytes(data[10:11], byteorder='little', signed=False)
    TD_range = (TD_mode_raw & 0b11110000)
    if TD_range & 0x80:
        print("Диапазон датчика 4095")
    if TD_range & 0b00000000:
         print("Диапазон датчика 1024")

    TD_flags = (TD_mode_raw & 0b00001111)
    if TD_flags & 0b00000000:
        print("Обычный алгоритм расчета")
    if TD_flags & 0b00000001:
        print("Алгоритм расчета с тарировочной таблицей")

      # Расшифровка сглаживания и термокомпенсации
    TD_smoth_and_copm_raw = int.from_bytes(data[11:12], byteorder='little', signed=False)
    TD_therm_compensation = (TD_smoth_and_copm_raw & 0b11110000)
    if TD_therm_compensation & 0x80:
         print("Термокомпенсация выключена")
    if TD_therm_compensation & 0b00000000:
          print("Термокомпенсация включена")

    TD_smoothing = TD_smoth_and_copm_raw & 0x0F
    print(f'Сглаживание: {TD_smoothing}')

In [13]:
timeout = 60

In [14]:
NAME = 'TD_383448'

In [15]:
class MyScanner:
    def __init__(self, timeout):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        self.scanning = asyncio.Event()
        self.address = None
        self.timeout = timeout

    def detection_callback(self, device, advertisement_data):
        if device.name == NAME: 
            print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, advertisement_data.rssi))
            adv_decrypt(advertisement_data.manufacturer_data[3862])

    async def run(self):
        await self._scanner.start()
        self.scanning.set()
        end_time = loop.time() + self.timeout
        while self.scanning.is_set():
            if loop.time() > end_time:
                self.scanning.clear()
                print('\t\tScan has timed out so we terminate')
            await asyncio.sleep(0.1)
        await self._scanner.stop()

    def get_address(self):
        return self.address


if __name__ == '__main__':
    my_scanner = MyScanner()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(my_scanner.run(timeout=timeout))
    address = my_scanner.get_address()

Device CA:17:37:73:61:98 (TD_383448), RSSI=-54 dB {} 2023-05-29 14:53:48.472742
Уровень топлива: 4095
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30684 
Термокомпенсация выключена
Сглаживание: 0
Device CA:17:37:73:61:98 (TD_383448), RSSI=-54 dB {} 2023-05-29 14:53:54.484297
Уровень топлива: 4095
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30684 
Термокомпенсация выключена
Сглаживание: 0
Device CA:17:37:73:61:98 (TD_383448), RSSI=-54 dB {} 2023-05-29 14:53:54.486296
Уровень топлива: 4095
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30684 
Термокомпенсация выключена
Сглаживание: 0
Device CA:17:37:73:61:98 (TD_383448), RSSI=-52 dB {} 2023-05-29 14:54:21.541888
Уровень топлива: 4095
Напряжение встроенной батареи: 3.7 В
Температура: 23 Градус Цельсия
Версия прошивки: 141
Период: 30684 
Термокомпенсация выключена
Сглаживание: 0
Device C

In [9]:
address

'CA:17:37:73:61:98'

In [10]:
#MESSAGE = b"SD,HK:1:30000,LK:1:10000\r"
MESSAGE = b"GA\r"

async def client(MESSAGE, timeout=60):
    async with BleakClient(address, timeout=timeout) as client:
        # Connect to the device and discover the services and characteristics
                        
        # Subscribe to notifications for the RX characteristic
        await client.start_notify(14, callback)
        
                        
        # Записываем сообщение в TX характеристику 
        await client.write_gatt_char(12, MESSAGE)
        print(f'WTIRE: {MESSAGE} on TX_char')

        await asyncio.sleep(1)
        
        # Stop receiving notifications
        await client.stop_notify(14)

async def callback(sender: int, data: bytearray):
    print(f'Received notification from RX_char: {data}')

loop = asyncio.get_event_loop()
loop.run_until_complete(client(MESSAGE))

WTIRE: b'GA\r' on TX_char
Received notification from RX_char: bytearray(b'AGA,UT:1:23,UL:1:4095,HK:1:30000,LK:1:10000,US:1:30684,SE:1:383448,VB:2:3.7,WM:1:32768,TS:1:2101499,')
Received notification from RX_char: bytearray(b'EM:1:0,TT:1:0,TM:1:10,UD:0:D8D9C5DDEEFF,VD:1:1,VV:1:141\r')
